In [1]:
import torch
from torch import nn
import numpy as np
import random
from collections import deque
from itertools import count
import torch.nn.functional as F
from mdp import TradeExecutionEnv, DiscreteTradeSizeWrapper
from tensorboardX import SummaryWriter
from torch.distributions import Categorical


SEED = 42
HORIZON = 5 * 12 * 2
UNITS_TO_SELL = 64
BATCH_SIZE = 32
EPOCHS = 300
EVAL_EPOCHS = 50
EPSILON = 0.1
GAMMA = 0.99

env = TradeExecutionEnv()

trade_sizes = {
  i: i*2 for i in range(33)
}
env = DiscreteTradeSizeWrapper(env, trade_sizes)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


class CategoricalPolicy(nn.Module):
    def __init__(self, num_states, num_actions, hidden_dim=32) -> None:
        super().__init__()
        self.fc1 = nn.Linear(num_states, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_actions)
        
    def forward(self, x):
        x = F.relu(self.fc1(x[:, -1, :]))
        x = F.softmax(self.fc2(x), dim=1)
        return x

    def select_action(self, state):
        probs = self.forward(state)
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

In [2]:
def parse_state(state):
    data = torch.FloatTensor(np.stack([
        state["low"].to_numpy(),
        state["high"].to_numpy(),
        state["close"].to_numpy(),
        state["open"].to_numpy(),
        state["volume"].to_numpy(),
    ])).T
    return torch.concat([
        data,
        torch.repeat_interleave(torch.FloatTensor([[state["units_sold"]]]), 6, 0),
        torch.repeat_interleave(torch.FloatTensor([[state["cost_basis"]]]), 6, 0),
        torch.repeat_interleave(torch.FloatTensor([[state["steps_left"]]]), 6, 0),
    ], dim=1).unsqueeze(0)

In [3]:
def sample_rollouts(policy, env, num_rollouts, seed=SEED):
    trajs = []
    for e in range(num_rollouts):
        tau = []
        state = env.reset(UNITS_TO_SELL, HORIZON, seed=(seed * num_rollouts) + e)
        state = parse_state(state)
        done = False
        while not done:
            with torch.no_grad():
                action, _ = policy.select_action(state)
            next_state, reward, done, _, _ = env.step(action)
            next_state = parse_state(next_state)
            tau.append((state, action, reward))
            state = next_state
        states, actions, rewards = zip(*tau)
        states = torch.cat(states)
        actions = torch.tensor([actions])
        rewards = torch.tensor([rewards])
        trajs.append((states, actions, rewards))
    return trajs

In [4]:
def R_tau(r, gamma):
    gammas = torch.tensor([gamma**i for i in range(len(r))])
    return torch.sum(gammas * r)

def avg_batch_rewards(trajs, gamma):
    return torch.mean(torch.stack([R_tau(r, gamma) for _, _, r in trajs]))

def grad_log_pi(policy, states, actions):
    logits = policy(states)
    log_probs = torch.log(logits)
    log_probs = log_probs.gather(1,actions).T.flatten()
    g = [torch.autograd.grad(log_p, policy.parameters(), retain_graph=True) for log_p in log_probs]
    return g

In [5]:
def grad_log_tau(policy, tau):
  g = grad_log_pi(policy, tau[0], tau[1])
  stacked_g = [torch.stack(g_).view(len(tau[0]), -1) for g_ in zip(*g)]
  return torch.cat(stacked_g, axis=1).sum(axis=0)

def grad_U_tau(policy, tau, gamma):
  return grad_log_tau(policy, tau) * R_tau(tau[2], gamma)


In [6]:
policy = CategoricalPolicy(len(env.observation_space), len(trade_sizes)).to(device)

train_rewards = []
for e in range(EPOCHS):
  trajs = sample_rollouts(policy, env, BATCH_SIZE, seed=e)
  g_u_tau = [grad_U_tau(policy, tau, GAMMA) for tau in trajs]
  Fish = torch.stack([gut.unsqueeze(1) @ gut.unsqueeze(0) for gut in g_u_tau]).mean(axis=0)
  g_u = torch.stack(g_u_tau).mean(axis=0)
  u = torch.linalg.pinv(Fish) @ g_u.unsqueeze(0).T
  #u = torch.linalg.inv(Fish + torch.FloatTensor(np.diag([0.1] * Fish.shape[0]))) @ g_u.unsqueeze(0).T
  g = u * torch.sqrt(2 * EPSILON / (g_u.T @ u))
  #print(f"G: {g}")
  n = 0
  for i, p in enumerate(policy.parameters()):
    num_elements = p.numel()
    p.data += g[n:n+num_elements].view(p.shape)
    n += num_elements
  policy.zero_grad()
  train_rewards.append(avg_batch_rewards(trajs, GAMMA).item())
  if e % 10 == 0:
    print(f"iter {e}: {avg_batch_rewards(trajs, GAMMA)}")

/var/folders/k3/xxcvpr8d71d8zr8__nlkymdm0000gn/T/ipykernel_56794/408031376.py:11: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3580.)
  g = u * torch.sqrt(2 * EPSILON / (g_u.T @ u))


iter 0: -9.707906789809204
iter 10: -10.0
iter 20: -9.393291799362299
iter 30: -8.508725680989519
iter 40: -10.0
iter 50: -9.705515914872356
iter 60: -9.691710885192952
iter 70: -8.489100294018947
iter 80: -9.12196761485022
iter 90: -9.09456244439741
iter 100: -9.41599968103855
iter 110: -8.818810472996935
iter 120: -8.194608088762617
iter 130: -9.68976557544274
iter 140: -8.512150796672536
iter 150: -9.688682870275528
iter 160: -9.401640201756003
iter 170: -9.106346155611117
iter 180: -9.392359980649863
iter 190: -9.422141135973249
iter 200: -8.492243488305343
iter 210: -8.18447905256977
iter 220: -10.0
iter 230: -9.100808011783618
iter 240: -8.48213370144627
iter 250: -9.075429974248003
iter 260: -8.48247106113919
iter 270: -9.112153707933196
iter 280: -9.099216905375275
iter 290: -9.413890219237118


In [7]:
same_mdp_eval_rewards = []
for e in range(EVAL_EPOCHS):
    state = env.reset(UNITS_TO_SELL, HORIZON, seed=SEED)
    state = parse_state(state)
    done = False
    episode_rewards = 0
    while not done:
        with torch.no_grad():
            action = torch.argmax(policy(state)).item()
        next_state, reward, done, _, _ = env.step(action)
        next_state = parse_state(next_state)
        state = next_state
        episode_rewards += reward
    same_mdp_eval_rewards.append(episode_rewards)
print(f"Avg same mdp eval reward: {np.mean(same_mdp_eval_rewards)}")

Avg same mdp eval reward: -10.0


In [8]:
train_mdp_eval_rewards = []
for e in range(EVAL_EPOCHS):
    state = env.reset(UNITS_TO_SELL, HORIZON, seed=(BATCH_SIZE * EPOCHS)+e)
    state = parse_state(state)
    done = False
    episode_rewards = 0
    while not done:
        with torch.no_grad():
            action = torch.argmax(policy(state)).item()
        next_state, reward, done, _, _ = env.step(action)
        next_state = parse_state(next_state)
        state = next_state
        episode_rewards += reward
    train_mdp_eval_rewards.append(episode_rewards)
print(f"Avg train mdp eval reward: {np.mean(train_mdp_eval_rewards)}")

Avg train mdp eval reward: -3.1852614945864657


In [9]:
test_mdp_eval_rewards = []
for e in range(EVAL_EPOCHS):
    state = env.reset(UNITS_TO_SELL, HORIZON, seed=SEED+e, test=True)
    state = parse_state(state)
    done = False
    episode_rewards = 0
    while not done:
        with torch.no_grad():
            action = torch.argmax(policy(state)).item()
        next_state, reward, done, _, _ = env.step(action)
        next_state = parse_state(next_state)
        state = next_state
        episode_rewards += reward
    test_mdp_eval_rewards.append(episode_rewards)
print(f"Avg test mdp eval reward: {np.mean(test_mdp_eval_rewards)}")

Avg test mdp eval reward: -2.186847392098957


In [10]:
import json

EXP_NAME = "natural_gradient_update_meta_mdp_train.json"
with open(f"./results/{EXP_NAME}", "w+") as f:
    json.dump({
        "train_rewards": train_rewards,
        "same_mdp_eval_rewards": same_mdp_eval_rewards,
        "train_mdp_eval_rewards": train_mdp_eval_rewards,
        "test_mdp_eval_rewards": test_mdp_eval_rewards
    }, f)